<a href="https://colab.research.google.com/github/tumblingdice512/Research/blob/master/model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
import jax
import numpy as np
import random

In [113]:
#モデルについて
#幅N、深さL+1(l=0,1,･･･,L)のネットワーク
#各パーセプトロンはM成分を持つスピンで各成分は+1か-1を取る
#第l層と第l+1層の各パーセプトロンの同一成分は全て結合→重みJ
#各層の重みJはΣJ**2 = Nと正規化されている
#第l層のスピンの値に重みをかけた値に対して符号関数をかませたものが第l+1層の値(0に対しては+1か-1にする)
#初期条件としては、入力層と出力層のスピンの値を与える
#入力層のスピンの値と、初期の重みの条件から、出力層を予想して、
#
#
#
#

In [114]:
N = 20
L = 5

In [115]:
a = [1, -1]
b = random.choice(a)

print(b)

1


In [116]:
#S0 = np.array([random.choice([1,-1]) for i in range(N)])
#SL = np.array([random.choice([1,-1]) for i in range(N)])

#print(S0)
#print("S0の要素数は",len(S0))

#print(SL)
#print("SLの要素数は",len(SL))

#初期条件として与えるスピンを生成
#各パーセプトロンのスピンの成分数を増やすときは、SOとSLを行列の形式にすればよい。

In [117]:
S0 = np.array([[random.choice([1,-1]) for i in range(N)],[random.choice([1,-1]) for i in range(N)]])
SL = np.array([[random.choice([1,-1]) for i in range(N)],[random.choice([1,-1]) for i in range(N)]])

print(S0)

print(SL)

#S0とSLの成分数が2の時

[[ 1  1  1 -1 -1  1  1  1  1  1 -1 -1 -1 -1  1  1  1 -1 -1 -1]
 [-1  1 -1  1 -1 -1 -1 -1 -1 -1  1 -1  1  1 -1 -1  1  1  1 -1]]
[[-1  1  1  1  1 -1 -1 -1  1 -1  1 -1  1  1  1 -1  1  1 -1  1]
 [ 1 -1 -1  1  1 -1  1  1  1 -1 -1 -1 -1 -1 -1 -1  1 -1  1  1]]


In [118]:
def MSE(t, y):
    mse = np.mean(np.sum(np.square(t-y),axis =1),axis = 0)
    return mse



In [128]:
A = np.array([[3,5,1],[4,12,1]])
print(A**2)
A_norm = (np.sum(np.square(A),axis=0))**(1/2)

A_normalized = A / A_norm

print(A_norm)
print(A_norm_2)

print(A_normalized)


[[  9  25   1]
 [ 16 144   1]]
[ 5.         13.          1.41421356]
[ 5.         13.          1.41421356]
[[0.6        0.38461538 0.70710678]
 [0.8        0.92307692 0.70710678]]


In [123]:
def weight_norm(x):
    x_norm = (np.sum(np.square(x),axis=0)**(1/2))
    return x_norm

In [124]:
class Network():
  #ネットワークを定義する
  #構成[入力層, 第1層, 第2層, 第3層, 第4層, 第5層]
  #全結合

  def __init__(self):
    #重みの定義
    self.w1 = np.random.randn(N,N)
    self.w2 = np.random.randn(N,N)
    self.w3 = np.random.randn(N,N)
    self.w4 = np.random.randn(N,N)
    self.w5 = np.random.randn(N,N)

  def forward(self, x):
    #重みの正規化
    self.J1 = self.w1 * (weight_norm(self.w1 + 1e-7))**(-1) * N
    self.J2 = self.w2 * (weight_norm(self.w2 + 1e-7))**(-1) * N
    self.J3 = self.w3 * (weight_norm(self.w3 + 1e-7))**(-1) * N
    self.J4 = self.w4 * (weight_norm(self.w4 + 1e-7))**(-1) * N
    self.J5 = self.w5 * (weight_norm(self.w5 + 1e-7))**(-1) * N

    self.layer0 = x
    self.layer1 = np.sign((np.dot(self.layer0, self.J1))+1e-7)
    self.layer2 = np.sign((np.dot(self.layer1, self.J2))+1e-7)
    self.layer3 = np.sign((np.dot(self.layer2, self.J3))+1e-7)
    self.layer4 = np.sign((np.dot(self.layer3, self.J4))+1e-7)
    self.out = np.sign((np.dot(self.layer4, self.J5))+1e-7)
    return self.out
    

  def backward(self, t, y):
    #誤差逆伝播
    delta5 = -2*(t-y)
    delta4 = np.dot(delta5,self.J5.T)
    delta3 = np.dot(delta4,self.J4.T) 
    delta2 = np.dot(delta3,self.J3.T)
    delta1 = np.dot(delta2,self.J2.T)

    #重みの勾配
    self.dedJ5 = np.dot(self.layer4.T, delta5) / delta5.shape[0]
    self.dedJ4 = np.dot(self.layer3.T, delta4) / delta4.shape[0]
    self.dedJ3 = np.dot(self.layer2.T, delta3) / delta3.shape[0]
    self.dedJ2 = np.dot(self.layer1.T, delta2) / delta2.shape[0]
    self.dedJ1 = np.dot(self.layer0.T, delta1) / delta1.shape[0]

  def optimize_GradientDecent(self, lr):
    #重みの更新    
    self.J1 -= lr * self.dedJ1
    self.J2 -= lr * self.dedJ2
    self.J3 -= lr * self.dedJ3
    self.J4 -= lr * self.dedJ4
    self.J5 -= lr * self.dedJ5

    #重みの正規化
    self.J1 = self.w1 * (weight_norm(self.w1 + 1e-7))**(-1) * N
    self.J2 = self.w2 * (weight_norm(self.w2 + 1e-7))**(-1) * N
    self.J3 = self.w3 * (weight_norm(self.w3 + 1e-7))**(-1) * N
    self.J4 = self.w4 * (weight_norm(self.w4 + 1e-7))**(-1) * N
    self.J5 = self.w5 * (weight_norm(self.w5 + 1e-7))**(-1) * N



In [125]:
model = Network()

# 学習率
lr = 0.01
# 学習エポック数
n_epoch = 200

# n_epoch繰り返す
for n in range(n_epoch):
    y = model.forward(S0)
    train_loss = MSE(SL, y)
    model.backward(SL, y)
    model.optimize_GradientDecent(lr)
    print('EPOCH ', n + 1, ' | TRAIN LOSS ',
          train_loss)

EPOCH  1  | TRAIN LOSS  34.0
EPOCH  2  | TRAIN LOSS  34.0
EPOCH  3  | TRAIN LOSS  34.0
EPOCH  4  | TRAIN LOSS  34.0
EPOCH  5  | TRAIN LOSS  34.0
EPOCH  6  | TRAIN LOSS  34.0
EPOCH  7  | TRAIN LOSS  34.0
EPOCH  8  | TRAIN LOSS  34.0
EPOCH  9  | TRAIN LOSS  34.0
EPOCH  10  | TRAIN LOSS  34.0
EPOCH  11  | TRAIN LOSS  34.0
EPOCH  12  | TRAIN LOSS  34.0
EPOCH  13  | TRAIN LOSS  34.0
EPOCH  14  | TRAIN LOSS  34.0
EPOCH  15  | TRAIN LOSS  34.0
EPOCH  16  | TRAIN LOSS  34.0
EPOCH  17  | TRAIN LOSS  34.0
EPOCH  18  | TRAIN LOSS  34.0
EPOCH  19  | TRAIN LOSS  34.0
EPOCH  20  | TRAIN LOSS  34.0
EPOCH  21  | TRAIN LOSS  34.0
EPOCH  22  | TRAIN LOSS  34.0
EPOCH  23  | TRAIN LOSS  34.0
EPOCH  24  | TRAIN LOSS  34.0
EPOCH  25  | TRAIN LOSS  34.0
EPOCH  26  | TRAIN LOSS  34.0
EPOCH  27  | TRAIN LOSS  34.0
EPOCH  28  | TRAIN LOSS  34.0
EPOCH  29  | TRAIN LOSS  34.0
EPOCH  30  | TRAIN LOSS  34.0
EPOCH  31  | TRAIN LOSS  34.0
EPOCH  32  | TRAIN LOSS  34.0
EPOCH  33  | TRAIN LOSS  34.0
EPOCH  34  | TRAIN 

In [126]:
print(y)
print(SL)

[[-1.  1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1.  1.  1.  1. -1.  1. -1.
   1.  1.]
 [ 1. -1.  1. -1.  1.  1.  1. -1. -1.  1. -1.  1. -1.  1. -1.  1.  1. -1.
  -1. -1.]]
[[-1  1  1  1  1 -1 -1 -1  1 -1  1 -1  1  1  1 -1  1  1 -1  1]
 [ 1 -1 -1  1  1 -1  1  1  1 -1 -1 -1 -1 -1 -1 -1  1 -1  1  1]]
